In [1]:
from src.utils.myOptimization import solveGroupProblem
from src.utils.myPrediction import generate_random_features, customPredictionModel
import numpy as np
import cvxpy as cp
import torch
import torch.nn as nn
import pandas as pd
from src.utils.features import get_all_features

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'solveGroupProblem' from 'src.utils.myOptimization' (/Users/dennis/Downloads/2024-fall/research/Fairness-Decision-Focused-Loss/Organized-FDFL/src/utils/myOptimization.py)

In [ ]:
df = pd.read_csv('E:\\User\\Stevens\\MyRepo\\Organized-FDFL\\src\\data\\data.csv')
df = df.sample(n=5000,random_state=1)

# Normalized cost 
cost = np.array(df['cost_t_capped'].values) 
cost = np.maximum(cost, 0.1) 
 

# All features, standardized
features = df[get_all_features(df)].values
scaler = StandardScaler()
features = scaler.fit_transform(features)

# True benefit, predictor label 
true_benefit = np.array(df['benefit'].values).reshape(-1, 1) * 100
true_benefit = np.maximum(true_benefit, 0.1) 


# Group labels, 0 is White (Majority), 1 is Black
race = np.array(df['race'].values).reshape(-1, 1)




In [ ]:
true_benefit.max(), true_benefit.min()

(81.98538295015143, 0.1)

In [ ]:
cost.max(), cost.min()

(1.0, 0.1)

In [ ]:
def alpha_fairness_group_utilities(benefit, allocation, group, alpha):
    """
    Compute group-wise alpha-fairness utilities.
    """
    groups = np.unique(group)
    utils = []
    for k in groups:
        mask = (group == k)
        Gk = float(mask.sum())
        # Compute average utility in group k
        util_k = (benefit[mask] * allocation[mask]).sum(axis=0).mean()  # mean total utility per individual in group
        if alpha == 1:
            val = np.log(util_k) if util_k > 0 else -np.inf
        elif alpha == 0:
            val = util_k
        elif alpha == float('inf'):
            # Min utility as min total utility)
            val = (benefit[mask] * allocation[mask]).sum(axis=0).min()
        else:
            val = util_k**(1 - alpha) / (1 - alpha)
        utils.append(val)
    return np.array(utils).sum()

In [ ]:
def run_prediction_and_optimization(n=4, T=1, n_features=3, n_groups=2, alpha=1.0, Q=5.0):
    # Step 1: Generate random data
    features, costs, groups, budget = generate_random_features(n, n_features, T, n_groups, Q)
    # Flatten features per individual and time for model input, shape (n*T, n_features)
    # Or aggregate features across time depending on your model design
    X = features.reshape(n * T, n_features)
    
    # Step 2: Predict benefit using a simple linear model
    # Convert to torch tensor
    X_torch = torch.tensor(X, dtype=torch.float32)
    model = nn.Linear(n_features, 1)  # simple linear regression
    with torch.no_grad():
        preds = model(X_torch).numpy().reshape(n, T)
    
    # Optional: ensure positivity of predicted benefits (important for alpha-fairness)
    preds = np.maximum(preds, 1e-3)
    
    # Step 3: Solve group problem
    allocation, obj_value = solveGroupProblem(preds, costs, groups, alpha, budget)
    
    # Step 4: Compute group-wise utility values
    group_utils = alpha_fairness_group_utilities(preds, allocation, groups, alpha)
    
    return {
        'predicted_benefit': preds,
        'allocation': allocation,
        'objective_value': obj_value,
        'group_utilities': group_utils,
        'groups': groups
    }

# Run example
result = run_prediction_and_optimization()
print("Predicted Benefit:\n", np.round(result['predicted_benefit'], 3))
print("Allocation:\n", np.round(result['allocation'], 3))
print("Objective value:", result['objective_value'])
print("Group utilities:", result['group_utilities'])
print("Groups:", result['groups'])

Predicted Benefit:
 [[0.001]
 [0.001]
 [0.03 ]
 [0.001]]
Allocation:
 [[ 0.   ]
 [ 0.   ]
 [14.473]
 [14.421]]
Objective value: -5.079654644712488
Group utilities: -5.079654644712488
Groups: [1 1 1 0]


In [ ]:
class FairRiskPredictor(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # Output layer
            nn.Linear(64, 1),
            nn.Softplus()
        )
            
    def forward(self, x):
        return self.model(x)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Convert the preprocessed features and true benefit to torch tensors
X_tensor = torch.tensor(features, dtype=torch.float32)
y_tensor = torch.tensor(true_benefit, dtype=torch.float32)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create a DataLoader for the training data
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=len(y_tensor), shuffle=True)

# Define a simple linear regression model
input_dim = X_train.shape[1]
# model = nn.Linear(input_dim, 1)
model = FairRiskPredictor(input_dim, dropout_rate=0.1)

# Use Mean Squared Error loss and Adam optimizer with weight decay to alleviate overfitting
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

# Train for a moderate number of epochs with early stopping hints through monitoring loss
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)
    avg_loss = running_loss / len(train_dataset)
    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

# With the trained model, generate predictions on the full dataset
model.eval()
with torch.no_grad():
    pred_benefit = model(X_tensor).numpy()

pred_benefit = np.maximum(pred_benefit, 1e-3)  # Ensure positivity
  # Ensure positivity of costs

# Solve the group problem with the predicted benefits


Epoch 20/30, Loss: 134.3057


1. I have PTO first half
2. Need to calculate `regret`
3. Train end-to-end.

In [ ]:
alpha = 2
Q = 1000

In [ ]:
sol, _ = solveGroupProblem(pred_benefit, cost, race, alpha=alpha, Q=Q)

true_sol, _ = solveGroupProblem(true_benefit, cost, race, alpha=alpha, Q=Q)
  # Ensure positivity
true_obj = alpha_fairness_group_utilities(true_benefit, true_sol, race, alpha=alpha)

print("True Objective Value:", true_obj)

pred_obj = alpha_fairness_group_utilities(true_benefit, sol, race, alpha=alpha)

print("Predicted Objective Value:", pred_obj)

c:\Users\14469\anaconda3\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


True Objective Value: -0.04606651691343278
Predicted Objective Value: -0.055021162113291666


In [ ]:
normalized_regret = (true_obj - pred_obj) / (abs(true_obj) + 1e-7)
print("Normalized Regret:", normalized_regret)

Normalized Regret: 0.19438469329506494


In [ ]:
true_benefit.shape
mask = race == 2
# per-group best benefit-cost ratio ρ_k and its argmax (i*,t*)
rho_k = np.zeros(2)
idx_k = np.zeros((2, 2), dtype=int)

In [ ]:
mask = race == 0
ratio = (true_benefit / cost)
ratio = ratio[mask].reshape(-1)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 25000000 but corresponding boolean dimension is 5000

In [ ]:
for i in range(2):
    mask = race == i
    print(mask)
    print(true_benefit[mask].shape, cost[mask].shape)
    ratio = (true_benefit[mask] / cost[mask]).reshape(-1)
    i_star = ratio.argmax()
    i_glob = np.where(mask)[0][i_star // true_benefit.shape[1]]
    t_glob = i_star % true_benefit.shape[1]
    rho_k[i] = ratio.max()
    idx_k[i] = [i_glob, t_glob]

[[ True]
 [ True]
 [ True]
 ...
 [ True]
 [False]
 [ True]]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
def closed_form_group_alpha(
    b_hat,
    cost,
    group,
    alpha, 
    Q
):
    """
    Parameters
    ----------
    b_hat : (N,T)  positive utilities
    cost  : (N,T)  positive costs
    group : (N,)   integers 0..K-1
    Q     : float  budget
    alpha : float >0 or 0 or 1 or np.inf
    Returns
    -------
    d_star : (N,T) optimal doses
    """
    N, T = b_hat.shape
    K = group.max() + 1

    # per-group best benefit-cost ratio ρ_k and its argmax (i*,t*)
    rho_k = np.zeros(K)
    idx_k = np.zeros((K, 2), dtype=int)
    for k in range(K):
        mask = group == k
        ratio = (b_hat[mask] / cost[mask]).reshape(-1)
        i_star = ratio.argmax()
        i_glob = np.where(mask)[0][i_star // T]
        t_glob = i_star % T
        rho_k[k] = ratio.max()
        idx_k[k] = [i_glob, t_glob]

    p_k = rho_k / np.bincount(group)  # p_k = ρ_k / G_k

    # ------------ Stage I: allocate budget B_k = x_k ------------
    if alpha == 0:  # utilitarian
        winners = np.flatnonzero(p_k == p_k.max())
        x = np.zeros(K)
        x[winners] = Q / len(winners)
    elif alpha == 1:  # logarithmic
        x = np.full(K, Q / K)
    elif alpha == np.inf:  # max-min
        inv = 1 / p_k
        x = Q * inv / inv.sum()
    else:  # generic  (0<α<∞, α≠1)
        weights = p_k ** (1 / alpha - 1)
        x = Q * weights / weights.sum()

    # ------------ Stage II: spend each x_k on its best item -----
    d_star = np.zeros_like(b_hat)
    for k in range(K):
        i, t = idx_k[k]
        d_star[i, t] = x[k] / cost[i, t]

    return d_star

In [ ]:
sol = closed_form_group_alpha(true_benefit, cost, race, alpha=alpha, Q=Q)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
def AlphaFairness(util, alpha):
    if isinstance(util, torch.Tensor):
        util = util.detach().cpu().numpy() if isinstance(util, torch.Tensor) else util
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha) / (1-alpha))

def solve_optimization(gainF, risk, cost, alpha, Q):
    gainF = gainF.detach().cpu().numpy() if isinstance(gainF, torch.Tensor) else gainF
    risk = risk.detach().cpu().numpy() if isinstance(risk, torch.Tensor) else risk
    cost = cost.detach().cpu().numpy() if isinstance(cost, torch.Tensor) else cost

    risk = risk.clip(0.001)
    gainF, risk, cost = gainF.flatten(), risk.flatten(), cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)

    if gainF.shape != risk.shape or risk.shape != cost.shape:
        raise ValueError("Dimensions of gainF, risk, and cost do not match")

    utils = cp.multiply(cp.multiply(gainF, risk), d)
    constraints = [d >= 0, cp.sum(cost * d) <= Q]

    if alpha == 'inf':
        t = cp.Variable()
        objective = cp.Maximize(t)
        constraints.append(utils >= t)
    elif alpha == 1:
        objective = cp.Maximize(cp.sum(cp.log(utils)))
    elif alpha == 0:
        objective = cp.Maximize(cp.sum(utils))
    else:
        objective = cp.Maximize(cp.sum(utils**(1-alpha)) / (1-alpha))

    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=False, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})

    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")

    optimal_decision = d.value
    optimal_value = AlphaFairness(optimal_decision * gainF * risk, alpha)

    return optimal_decision, optimal_value
import numpy as np

def solve_closed_form(g, r, c, alpha, Q):

    g = g.detach().cpu().numpy() if isinstance(g, torch.Tensor) else g
    r = r.detach().cpu().numpy() if isinstance(r, torch.Tensor) else r
    c = c.detach().cpu().numpy() if isinstance(c, torch.Tensor) else c
    if c.shape != r.shape or c.shape != g.shape:
        raise ValueError("c, r, and g must have the same shape.")
    if np.any(c <= 0):
        raise ValueError("All cost values must be positive.")
    if np.any(r <= 0):
        raise ValueError("All risk values must be positive.")
    if np.any(g <= 0):
        raise ValueError("All gain factors must be positive.")
    
    n = len(c)
    utility = r * g
    
    if alpha == 0:
        ratios = utility / c
        sorted_indices = np.argsort(-ratios)  # Descending order
        d_star_closed = np.zeros(n)
        d_star_closed[sorted_indices[0]] = Q / c[sorted_indices[0]]
        
    elif alpha == 1:
        d_star_closed = Q / (n * c)
    
    elif alpha == 'inf':
        d_star_closed = (Q * c) / (utility * np.sum(c * c / utility))
        
    else:
        if alpha <= 0:
            raise ValueError("Alpha must be positive for general case.")
        #
        # d_i* = (c_i^(-1/alpha) * (r_i*g_i)^(1/alpha - 1) * Q) / sum_j(c_j^(-1/alpha) * (r_j*g_j)^(1/alpha - 1))
        
        numerator = np.power(c, -1/alpha) * np.power(utility, 1/alpha - 1)
        denominator = np.sum(numerator)
        
        if denominator == 0:
            raise ValueError("Denominator is zero in closed-form solution.")
            
        d_star_closed = (numerator / denominator) * Q
    
    # if not np.isclose(np.sum(c * d_star_closed), Q, rtol=1e-5):
    #     raise ValueError("Solution does not satisfy budget constraint.")
    obj = AlphaFairness(d_star_closed * utility, alpha)
        
    return d_star_closed, obj

def compute_gradient_closed_form(g, r, c, alpha, Q):
    """
    Compute the analytical gradient of the optimal solution with respect to r.

    This function computes the gradient matrix where each element (i, k) is the partial derivative
    of d_i* with respect to r_k.

    Parameters:
    - g (np.ndarray): Gain factors (g_i), shape (n,)
    - r (np.ndarray): Risk values (r_i), shape (n,)
    - c (np.ndarray): Cost values (c_i), shape (n,)
    - alpha (float or str): Fairness parameter. Can be 0, 1, 'inf', or a positive real number.
    - Q (float): Total budget.

    Returns:
    - gradient (np.ndarray): Gradient matrix of shape (n, n)
    """
    if alpha == 1:
        S = np.sum(c / (r * g))

    if alpha == 0:
        # Utilitarian case: Allocate everything to the individual with the highest ratio
        ratios = (r * g) / c
        i_star = np.argmax(ratios)
        # Gradient is Q * g_i / c_i at the allocated index, zero elsewhere
        gradient[i_star, i_star] = Q * g[i_star] / c[i_star]
        return gradient

    elif alpha == 'inf':
        # Maximin case
        n = len(c)
        utility = r * g  # Shape: (n,)
        S = np.sum(c**2 / utility)  # Scalar

        # Compute d_star
        d_star, _ = solve_closed_form(g,r,c, alpha='inf', Q=Q)  # Shape: (n,)

        # Initialize gradient matrix
        gradient = np.zeros((n, n))

        for i in range(n):
            for k in range(n):
                if i == k:
                    # ∂d_i*/∂r_i = -d_i*/r_i - (d_i* * c_i) / (r_i * g_i * S)
                    gradient[i, k] = -d_star[i] / r[i] - (d_star[i] * c[i]) / (r[i] * g[i] * S)
                else:
                    # ∂d_i*/∂r_k = (d_i* * c_k^2) / (c_i * r_k^2 * g_k * S)
                    gradient[i, k] = (d_star[i] * c[k]**2) / (c[i] * r[k]**2 * g[k] * S)
        return gradient

    else:
        # General alpha case
        if not isinstance(alpha, (int, float)):
            raise TypeError("Alpha must be a positive real number, 0, 1, or 'inf'.")
        if alpha <= 0:
            raise ValueError("Alpha must be positive for gradient computation.")

        # Compute the optimal decision variables
        d_star, _ = solve_closed_form(g, r, c, alpha, Q)  # Shape: (n,)

        # Compute the term (1/alpha - 1) * g / r
        term = (1.0 / alpha - 1.0) * g / r  # Shape: (n,)

        # Compute the outer product for off-diagonal elements
        # Each element (i, k) = -d_star[i] * d_star[k] * term[k] / Q
        gradient = -np.outer(d_star, d_star * term) / Q  # Shape: (n, n)

        # Compute the diagonal elements
        # Each diagonal element (i, i) = d_star[i] * term[i] * (1 - d_star[i]/Q)
        diag_elements = d_star * term * (1 - d_star / Q)  # Shape: (n,)

        # Set the diagonal elements
        np.fill_diagonal(gradient, diag_elements)

        return gradient

In [ ]:
true_r = true_benefit
pred_r = pred_benefit
gainF = np.ones_like(true_r)

sol, obj = solve_closed_form(gainF, true_r, cost, alpha=2, Q=100)
sol, obj

ValueError: c, r, and g must have the same shape.

In [ ]:
sol, obj = solve_optimization(gainF, true_r, cost, alpha=2, Q=100)
sol, obj

c:\Users\14469\anaconda3\Lib\site-packages\cvxpy\expressions\expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)


SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.